In [1]:
import os
import re
import requests
import zipfile
import pandas as pd
from functools import reduce

import nltk
from nltk.corpus import stopwords

# typing
from typing import List, Callable, Dict

# Data pipeline
The goal of this section is to convert initial textual input into a numerical format that is compatible with our models.

## Data Loading
Download the dataset and save it to file system.
The dataset is composed by 3 csv files: train, validation and test. These 3 csv files will be loaded directly into 3 different dataframes.

In [2]:
def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

def download_data(data_path):
    toy_data_path = os.path.join(data_path, 'fever_data.zip')
    toy_data_url_id = "1wArZhF9_SHW17WKNGeLmX-QTYw9Zscl1"
    toy_url = "https://docs.google.com/uc?export=download"

    if not os.path.exists(data_path):
        os.makedirs(data_path)

    if not os.path.exists(toy_data_path):
        print("Downloading FEVER data splits...")
        with requests.Session() as current_session:
            response = current_session.get(toy_url,
                                   params={'id': toy_data_url_id},
                                   stream=True)
        save_response_content(response, toy_data_path)
        print("Download completed!")

        print("Extracting dataset...")
        with zipfile.ZipFile(toy_data_path) as loaded_zip:
            loaded_zip.extractall(data_path)
        print("Extraction completed!")

download_data('dataset')

Load the files into different data frames.

In [3]:
# Be sure all columns have a name
column_names = ['Row', 'Claim', 'Evidence', 'ID', 'Label']

# Load the datasets
df_train = pd.read_csv ('dataset/train_pairs.csv', names=column_names, header=0)
df_val = pd.read_csv ('dataset/val_pairs.csv', names=column_names, header=0)
df_test = pd.read_csv ('dataset/test_pairs.csv', names=column_names, header=0)


In [4]:
df_train.head()

,Row,Claim,Evidence,ID,Label
0,0,Chris Hemsworth appeared in A Perfect Getaway.,2\tHemsworth has also appeared in the science ...,3,SUPPORTS
1,1,Roald Dahl is a writer.,0\tRoald Dahl -LRB- -LSB- langpronˈroʊ.əld _ ˈ...,7,SUPPORTS
2,2,Roald Dahl is a governor.,0\tRoald Dahl -LRB- -LSB- langpronˈroʊ.əld _ ˈ...,8,REFUTES
3,3,Ireland has relatively low-lying mountains.,10\tThe island 's geography comprises relative...,9,SUPPORTS
4,4,Ireland does not have relatively low-lying mou...,10\tThe island 's geography comprises relative...,10,REFUTES


## Data Pre-processing
Perform text cleaning and tokenization operations.
Start by creating some utility methods that will be used for cleaning the text.

In [5]:
# Special characters to remove: /(){}[]|@,;
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')

# Accepted symbols:
# - numbers between 0-9
# - all lower cased letters
# - whitespace, #, + _
GOOD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

# - ^ begnning of a string
# - \d any digit
# - \s whitespaces and tabs
BEGINNING_IDS_RE = re.compile('^\d*\s*')

# The stopwords are a list of words that are very very common but don’t 
# provide useful information for most text analysis procedures.
# Therefore, they will be removed from the dataset
try:
    STOPWORDS = set(stopwords.words('english'))
except LookupError:
    nltk.download('stopwords')
    STOPWORDS = set(stopwords.words('english'))

def replace_special_characters(text: str) -> str:
    """
    Replaces special characters, such as paranthesis,
    with spacing character
    """

    return REPLACE_BY_SPACE_RE.sub(' ', text)

def lower(text: str) -> str:
    """
    Transforms given text to lower case.
    Example:
    Input: 'I really like New York city'
    Output: 'i really like new your city'
    """

    return text.lower()

def replace_special_characters(text: str) -> str:
    """
    Replaces special characters, such as paranthesis,
    with spacing character
    """

    return REPLACE_BY_SPACE_RE.sub(' ', text)

def filter_out_uncommon_symbols(text: str) -> str:
    """
    Removes any special character that is not in the
    good symbols list (check regular expression)
    """

    return GOOD_SYMBOLS_RE.sub('', text)

def remove_stopwords(text: str) -> str:
    """
    Method used for removing most common words

    Parameters
    ----------
    text : str
        The text to process
    
    Returns
    -------
    text : str
        The processed text.
    """
    return ' '.join([x for x in text.split() if x and x not in STOPWORDS])

def strip_text(text: str) -> str:
    """
    Removes any left or right spacing (including carriage return) from text.
    Example:
    Input: '  This assignment is cool\n'
    Output: 'This assignment is cool'
    """

    return text.strip()

def replace_ids(text: str) -> str:
    """
    Method used for removing ids and some whitespaces that could appear
    at the beginning of the text.

    Parameters
    ----------
    text : str
        The text to process
    
    Returns
    -------
    text : str
        The processed text.
    """
    return BEGINNING_IDS_RE.sub('', text)

GENERIC_PREPROCESSING_PIPELINE = [
                                  lower,
                                  replace_special_characters,
                                  filter_out_uncommon_symbols,
                                  remove_stopwords,
                                  strip_text
                                  ]

EVIDENCES_PREPROCESSING_PIPELINE = GENERIC_PREPROCESSING_PIPELINE
EVIDENCES_PREPROCESSING_PIPELINE.insert(0, replace_ids)
    

def text_prepare(text: str,
                 filter_methods: List[Callable[[str], str]] = None) -> str:
    """
    Applies a list of pre-processing functions in sequence (reduce).
    Note that the order is important here!
    """
    filter_methods = \
        filter_methods if filter_methods is not None else GENERIC_PREPROCESSING_PIPELINE

    return reduce(lambda txt, f: f(txt), filter_methods, text)

Now we are ready to pre-process the dataset.

In [6]:
def preprocess_dataset(df: pd.DataFrame) -> pd.DataFrame:
    # Replace each sentence with its pre-processed version
    df['Evidence'] = df['Evidence'].apply(
        lambda txt: text_prepare(txt, filter_methods=EVIDENCES_PREPROCESSING_PIPELINE)
        )
    df['Claim'] = df['Claim'].apply(lambda txt: text_prepare(txt))
    
    return df

df_train = preprocess_dataset(df_train)
df_val = preprocess_dataset(df_val)
df_test = preprocess_dataset(df_test)

In [7]:
df_train.head()

,Row,Claim,Evidence,ID,Label
0,0,chris hemsworth appeared perfect getaway,hemsworth also appeared science fiction action...,3,SUPPORTS
1,1,roald dahl writer,roald dahl lrb lsb langpronrold _ dl rsb lsb u...,7,SUPPORTS
2,2,roald dahl governor,roald dahl lrb lsb langpronrold _ dl rsb lsb u...,8,REFUTES
3,3,ireland relatively lowlying mountains,island geography comprises relatively lowlying...,9,SUPPORTS
4,4,ireland relatively lowlying mountains,island geography comprises relatively lowlying...,10,REFUTES
